<a href="https://colab.research.google.com/github/ajaysaikiran2208/PyTorch/blob/main/Yield_Prediction_using%20gradient%20descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear Regression using PyTorch

Estimation of Crop Yield

In [1]:
import numpy as np
import torch

Training data


We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Linear regression model from scratch

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1

yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.2942,  0.7090,  0.4133],
        [ 0.3486,  0.6356,  1.0690]], requires_grad=True)
tensor([-1.3478, -0.5417], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 42.4510, 113.4616],
        [ 60.7239, 155.5335],
        [ 92.0366, 176.9639],
        [ 14.4221, 101.9035],
        [ 75.3488, 159.3621]], grad_fn=<AddBackward0>)


In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Loss function

In [10]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(1478.0931, grad_fn=<DivBackward0>)


Compute gradients

In [12]:
# Compute gradients
loss.backward()

In [13]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.2942,  0.7090,  0.4133],
        [ 0.3486,  0.6356,  1.0690]], requires_grad=True)
tensor([[-1572.1780, -1857.1061, -1132.0228],
        [ 4273.0493,  4053.5293,  2627.1350]])


Adjust weights and biases to reduce the loss

In [14]:
w
w.grad

tensor([[-1572.1780, -1857.1061, -1132.0228],
        [ 4273.0493,  4053.5293,  2627.1350]])

In [15]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [16]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(1478.0931, grad_fn=<DivBackward0>)


In [17]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Train the model using gradient descent


As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

In [18]:
# Generate predictions
preds = model(inputs)
print(preds)
print(inputs)

tensor([[ 45.3299, 106.4962],
        [ 64.5136, 146.3961],
        [ 96.5497, 166.2903],
        [ 17.2433,  94.8295],
        [ 79.0091, 150.6828]], grad_fn=<AddBackward0>)
tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])


In [19]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(1033.6937, grad_fn=<DivBackward0>)


In [20]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-1274.5829, -1536.3414,  -934.2849],
        [ 3555.2505,  3285.8459,  2152.6035]])
tensor([-15.6709,  40.9390])


In [21]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [22]:
print(w)
print(b)

tensor([[-0.2658,  0.7430,  0.4340],
        [ 0.2704,  0.5622,  1.0212]], requires_grad=True)
tensor([-1.3475, -0.5426], requires_grad=True)


In [23]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(733.7550, grad_fn=<DivBackward0>)


In [25]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(11.2331, grad_fn=<DivBackward0>)


In [27]:
# Predictions
preds

tensor([[ 57.4956,  71.4423],
        [ 81.3708, 102.0252],
        [120.0590, 128.0530],
        [ 24.2061,  44.1933],
        [ 98.3767, 117.1985]], grad_fn=<AddBackward0>)

In [28]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])